In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install keytotext
!pip install yake
!pip install rouge_score

In [3]:
import pandas as pd
import string
import numpy as np
import json
import torch
import tensorflow as tf

In [4]:
data_dir = "/content/drive/Shareddrives/PFE artists/title_generator/title_data.txt"
output_dir = "/content/drive/Shareddrives/PFE artists/Roxane/title_gen/"

In [5]:
data = pd.read_csv(data_dir)

In [6]:
print(data.shape)
data = data.rename(columns={"Titre":"text"})
data.head()

(321248, 1)


,text
0,Etude de chasseurs et guerriers
1,Deux enfants nus tenant leurs mains sur leur tête
2,"La tête de la Vierge, les yeux baissés"
3,"Paysage avec des maisons fortifiées, un cavali..."
4,"Paysage avec un moulin, une maison entourée d'..."


In [7]:
# créer dataframe de 2 columns : keywords, text
import yake
n_keywords_per_title = 3
extractor = yake.KeywordExtractor(lan="fr", n=1, top=n_keywords_per_title)

def extraction(text):
  doublets = extractor.extract_keywords(text)
  keywords = [w for (w,_) in doublets]
  # il faut exactement le même nombre de keywords pour chaque titre
  # on rajoute des keywords vides pour avoir une liste de la bonne taille
  while len(keywords) < n_keywords_per_title:
    keywords.append(" ")
  return keywords

data_s = data[:30000]
data_s["keywords"] = data_s["text"].astype("str").apply(extraction)
data_s.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,text,keywords
0,Etude de chasseurs et guerriers,"[Etude, guerriers, chasseurs]"
1,Deux enfants nus tenant leurs mains sur leur tête,"[tête, enfants, nus]"
2,"La tête de la Vierge, les yeux baissés","[Vierge, baissés, tête]"
3,"Paysage avec des maisons fortifiées, un cavali...","[Paysage, fortifiées, cavalier]"
4,"Paysage avec un moulin, une maison entourée d'...","[Paysage, moulin, personnages]"


In [8]:
# fin max tokens
def count_tokens(text):
  liste = text.split(" ")
  return len(liste)

liste_counts = data.text.apply(count_tokens)
m = max(liste_counts)
print(m)

86


In [9]:
import sklearn
data_s = sklearn.utils.shuffle(data_s)
# data_s = data_shuffled[:3000]
test_split = int(0.1*data_s.shape[0])
test_df = data_s[:test_split]
train_df = data_s[test_split:]

In [10]:
torch.cuda.empty_cache()
!PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256
print(torch.cuda.is_available())
print(tf.test.gpu_device_name())

True
/device:GPU:0


In [11]:
from keytotext import pipeline, trainer

# model = trainer()
# model.from_pretrained(model_name="t5-small")

# model.train(
#         train_df,
#         test_df,
#         source_max_token_len = 90,
#         target_max_token_len = 10,
#         batch_size = 32,
#         max_epochs = 10,
#         use_gpu = True,
#         outputdir=output_dir,
#         early_stopping_patience_epochs = 2,  # 0 to disable early stopping feature
#         # test_split=0.1,
#         # tpu_cores = None,
#     )
# model.save_model(model_dir=output_dir)

Global seed set to 42


In [14]:
model = trainer()
model.load_model(model_dir=output_dir)
input = ["cheval", "orange", "ciel"]
pred = model.predict(keywords=input, max_length=10, num_return_sequences=3, use_gpu=False, num_beams=3)
print(pred)
# model.evaluate(test_df, metrics= "rouge") # pour évaluer les métriques du modèle

Un cheval orange ciel et un ciel


KeyError: ignored

# Detection d'objets dans le tableau